# Project 2


# Movie Genre Classification

Classify a movie genre based on its plot.

<img src="https://github.com/andresmarin94/Repositorio_Personal/blob/master/moviegenre.png?raw=1"
     style="float: left; margin-right: 10px;" />




https://www.kaggle.com/c/miia4200-20191-p2-moviegenreclassification/overview


### Data

Input:
- movie plot

Output:
Probability of the movie belong to each genre


### Evaluation

- 20% API
- 30% Create a solution using with a Machine Learning algorithm - Presentation (5 slides)
- 50% Performance in the Kaggle competition (Normalized acording to class performance in the private leaderboard)


### Acknowledgements

We thank Professor Fabio Gonzalez, Ph.D. and his student John Arevalo for providing this dataset.

See https://arxiv.org/abs/1702.01992

## Sample Submission

In [109]:
import pandas as pd
import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier,BaggingClassifier
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')
import xgboost
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [110]:
import re

In [111]:
dataTraining = pd.read_csv('https://github.com/albahnsen/AdvancedMethodsDataAnalysisClass/raw/master/datasets/dataTraining.zip', encoding='UTF-8', index_col=0)
dataTesting = pd.read_csv('https://github.com/albahnsen/AdvancedMethodsDataAnalysisClass/raw/master/datasets/dataTesting.zip', encoding='UTF-8', index_col=0)

### Create count vectorizer


In [112]:
dataTraining.head(5)

,year,title,plot,genres,rating
3107,2003,Most,most is the story of a single father who takes...,"['Short', 'Drama']",8.0
900,2008,How to Be a Serial Killer,a serial killer decides to teach the secrets o...,"['Comedy', 'Crime', 'Horror']",5.6
6724,1941,A Woman's Face,"in sweden , a female blackmailer with a disfi...","['Drama', 'Film-Noir', 'Thriller']",7.2
4704,1954,Executive Suite,"in a friday afternoon in new york , the presi...",['Drama'],7.4
2582,1990,Narrow Margin,"in los angeles , the editor of a publishing h...","['Action', 'Crime', 'Thriller']",6.6


In [113]:
dataTesting.head(5)

,year,title,plot
1,1999,Message in a Bottle,"who meets by fate , shall be sealed by fate ...."
4,1978,Midnight Express,"the true story of billy hayes , an american c..."
5,1996,Primal Fear,martin vail left the chicago da ' s office to ...
6,1950,Crisis,husband and wife americans dr . eugene and mr...
7,1959,The Tingler,the coroner and scientist dr . warren chapin ...


In [114]:
dataTraining['title']=dataTraining['title'].str.lower()
dataTraining['title']=dataTraining['title'].str.replace(',', '') 
dataTraining['title']=dataTraining['title'].str.replace('.', '') 
dataTraining['title']=dataTraining['title'].str.replace('-', '') 
dataTraining['title']=dataTraining['title'].str.replace('_', '') 
dataTraining['plot']=dataTraining['plot'].str.lower()
dataTraining['plot']=dataTraining['plot'].str.replace(',', '') 
dataTraining['plot']=dataTraining['plot'].str.replace('.', '') 
dataTraining['plot']=dataTraining['plot'].str.replace('-', '') 
dataTraining['plot']=dataTraining['plot'].str.replace('_', '') 

In [115]:
dataTesting['plot']=dataTesting['plot'].str.lower()
dataTesting['plot']=dataTesting['plot'].str.replace(',', '') 
dataTesting['plot']=dataTesting['plot'].str.replace('.', '') 
dataTesting['plot']=dataTesting['plot'].str.replace('-', '') 
dataTesting['plot']=dataTesting['plot'].str.replace('_', '')
dataTesting['title']=dataTesting['title'].str.lower()
dataTesting['title']=dataTesting['title'].str.replace(',', '') 
dataTesting['title']=dataTesting['title'].str.replace('.', '') 
dataTesting['title']=dataTesting['title'].str.replace('-', '') 
dataTesting['title']=dataTesting['title'].str.replace('_', '') 

In [116]:
wordnet_lemmatizer = WordNetLemmatizer()

In [117]:
dataTraining['uno']= dataTraining['title']+dataTraining['plot']
dataTesting['uno'] = dataTesting['title']+dataTesting['plot']

data = dataTraining['uno'].append(dataTesting['uno'])
years = dataTraining['year'].append(dataTesting['year'])

In [118]:
wordnet_lemmatizer = WordNetLemmatizer()
def split_into_lemmas(text):
    text = text.lower()
    words = text.split()
    return [wordnet_lemmatizer.lemmatize(word, 's') for word in words]

vect = TfidfVectorizer(analyzer=split_into_lemmas, min_df=2,max_features=None,sublinear_tf=True,
                       strip_accents='unicode',ngram_range=(1,2),stop_words='english')

#X_dtm = vect.fit_transform(dataTraining['uno'],dataTraining['year'])
#X_dtm.shape
X_dtm = vect.fit_transform(dat, years)

In [119]:
X_dtm_test = X_dtm[dataTraining.shape[0]:,:]
X_dtm = X_dtm[:dataTraining.shape[0],:]

### Create y

In [120]:
dataTraining['genres'] = dataTraining['genres'].map(lambda x: eval(x))

le = MultiLabelBinarizer()
y_genres = le.fit_transform(dataTraining['genres'])

In [121]:
X_train, X_test, y_train_genres, y_test_genres = train_test_split(X_dtm, y_genres, test_size=0.3, random_state=666)

### Train multi-class multi-label model

In [122]:

#clf = OneVsRestClassifier(RandomForestClassifier(n_jobs=-1,max_depth=10,n_estimators=1000, random_state=60,criterion='entropy'))

In [125]:
LogReg_Model = OneVsRestClassifier(LogisticRegression(multi_class='multinomial',solver='newton-cg',C=1.5,
#                                                     warm_start=True,penalty='elasticnet',random_state=456, cv=10))
                                                      warm_start=True, penalty='l2',random_state=999), n_jobs=-1)

LogReg_Model.fit(X_train, y_train_genres)
y_pred_genres = LogReg_Model.predict_proba(X_test)
roc_auc_score(y_test_genres, y_pred_genres, average='macro')

0.892996649748475

### Predict the testing dataset

In [72]:


cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

y_pred_test_genres = LogReg_Model.predict_proba(X_dtm_test)


In [73]:
res = pd.DataFrame(y_pred_test_genres, index=dataTesting.index, columns=cols)

In [74]:
res.head()

,p_Action,p_Adventure,p_Animation,p_Biography,p_Comedy,p_Crime,p_Documentary,p_Drama,p_Family,p_Fantasy,p_Film-Noir,p_History,p_Horror,p_Music,p_Musical,p_Mystery,p_News,p_Romance,p_Sci-Fi,p_Short,p_Sport,p_Thriller,p_War,p_Western
1,0.101146,0.052219,0.018949,0.019606,0.335807,0.097615,0.011056,0.429440,0.032998,0.087134,0.017254,0.013329,0.065235,0.024723,0.030825,0.089718,0.000892,0.623165,0.030021,0.008591,0.014258,0.179302,0.014223,0.017395
4,0.111170,0.024148,0.024704,0.101792,0.280955,0.454654,0.032863,0.767957,0.040068,0.028899,0.011924,0.019897,0.064061,0.022788,0.019233,0.057342,0.001192,0.078811,0.018769,0.009760,0.020952,0.257412,0.029807,0.013684
5,0.063218,0.020198,0.006814,0.035531,0.103432,0.601081,0.012757,0.709453,0.013075,0.022921,0.060887,0.015168,0.047404,0.015353,0.018097,0.282213,0.000857,0.178762,0.028264,0.004595,0.020072,0.543169,0.018977,0.012721
6,0.128107,0.076501,0.009651,0.064344,0.166832,0.051222,0.010138,0.799170,0.030315,0.044062,0.027189,0.041367,0.094759,0.010138,0.016957,0.103032,0.000902,0.193123,0.136207,0.005871,0.025548,0.439140,0.071721,0.019029
7,0.031878,0.049184,0.022719,0.031619,0.193681,0.094783,0.017515,0.465777,0.041179,0.125501,0.036879,0.014400,0.520261,0.019135,0.024395,0.082721,0.000837,0.109624,0.272559,0.008709,0.008197,0.364447,0.013003,0.013418


In [75]:
res.to_csv('TFDF_LR.csv', index_label='ID')